In [ ]:
!nvidia-smi

Sun Sep  6 05:38:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
prepro = transforms.ToTensor()
tensor_to_pil = transforms.ToPILImage()

In [ ]:
mnist = datasets.MNIST(root='./MNIST',
                       train=True,
                       transform=prepro,
                       download=True)

Extracting ./MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# data preprocess
transform=transforms.Compose([
                transforms.Resize(32) ,
                transforms.ToTensor(),
                transforms.Normalize([0.5],[0.5])
                ])
dataloader=torch.utils.data.DataLoader(datasets.MNIST('data', train=True, download=True, transform=transform),
                                   32,
                                   shuffle = True,
                                   num_workers=2)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


In [ ]:
image, label = mnist[0]

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size = 10,
                                          shuffle = True,
                                          num_workers = 2)

In [ ]:
data_iter = iter(data_loader)

In [ ]:
images, labels = data_iter.next()

In [ ]:
class Generator(nn.Module):
  def __init__(self, z_dim=100): # dimension of latent vector
    super(Generator,self).__init__()
    self.Conv2d1_1 = nn.ConvTranspose2d(z_dim,256,4,1,0,bias=False);
    self.BN1_1 = nn.BatchNorm2d(256);
    self.Conv2d1_2 = nn.ConvTranspose2d(10,256,4,1,0,bias=False);
    self.BN1_2 = nn.BatchNorm2d(256);
    self.Conv2d2 = nn.ConvTranspose2d(512,256,4,2,1,bias=False);
    self.BN2 = nn.BatchNorm2d(256);
    self.Conv2d3 = nn.ConvTranspose2d(256,128,4,2,1,bias=False);
    self.BN3 = nn.BatchNorm2d(128);
    self.Conv2d4 = nn.ConvTranspose2d(128,1,4,2,1,bias=False);
  
  def forward(self,z,label):
    h = F.relu(self.BN1_1(self.Conv2d1_1(z)))
    y = F.relu(self.BN1_2(self.Conv2d1_2(label)))
    h = torch.cat([h,y],1);
    h = F.relu(self.BN2(self.Conv2d2(h)))
    h = F.relu(self.BN3(self.Conv2d3(h)))
    image = F.tanh(self.Conv2d4(h))
    
    return image

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.Conv2d1_1 = nn.Conv2d(1,64,4,2,1,bias=False);
    self.Conv2d1_2 = nn.Conv2d(10,64,4,2,1,bias=False);
    self.Conv2d3 = nn.Conv2d(128,256,4,2,1,bias=False);
    self.BN3 = nn.BatchNorm2d(256);
    self.Conv2d4 = nn.Conv2d(256,512,4,2,1,bias=False);
    self.BN4 = nn.BatchNorm2d(512);
    self.Conv2d5 = nn.Conv2d(512,1,4,1,0,bias=False);

  def forward(self, f_image,label):
    h = F.leaky_relu(self.Conv2d1_1(f_image))
    y = F.leaky_relu(self.Conv2d1_2(label))
    h = torch.cat([h,y],1)
    h = F.leaky_relu(self.BN3(self.Conv2d3(h)))
    h = F.leaky_relu(self.BN4(self.Conv2d4(h)))
    prob_real = F.sigmoid(self.Conv2d5(h))

    return prob_real

In [ ]:
G = Generator()

In [ ]:
D = Discriminator()

In [ ]:
z = Variable(torch.randn(32,100,1,1))

In [ ]:
G.cuda()
D.cuda()

Discriminator(
  (Conv2d1_1): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (Conv2d1_2): Conv2d(10, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (Conv2d3): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (BN3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Conv2d4): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (BN4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Conv2d5): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
)

In [ ]:
# Define optimizer
d_optimizer = torch.optim.Adam(D.parameters(),lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(),lr=0.0002)

In [ ]:
batch_size = 32
z_dim = 100

In [ ]:
criterion = nn.BCELoss()

In [ ]:
D.cuda()
G.cuda()

Generator(
  (Conv2d1_1): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (BN1_1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Conv2d1_2): ConvTranspose2d(10, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (BN1_2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Conv2d2): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (BN2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Conv2d3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (BN3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (Conv2d4): ConvTranspose2d(128, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
)

In [ ]:
# BCE Loss =  -y*log(x)-(1-y)*log(1-x)
# y = 1 -> -log(x)
# y = 0 -> -log(1-x)

logD(x)가 최대가 되어야한다.
Real image에 대해서는 0부터 1사이의 probability 값 중 당연히 1에 가까운 값이 되어야한다.
log함수의 경우 0~1의 x축에서 1에 가까워질수록 커짐.
Maximization -> -를 붙여줘야한다 (optimization은 기본적으로 minimize라)

Fake image 0에 가까운 값. log(1-x)는 0에서 최대.

In [ ]:
for epoch in range(100):
  for i, (images, labels) in enumerate(dataloader):
    ### Train D ###
    images = Variable(images).cuda()
    labels = Variable(labels).cuda()
    z = Variable(torch.randn(batch_size, z_dim,1,1)).cuda()

    real_labels = Variable(torch.ones(batch_size)).cuda()
    fake_labels = Variable(torch.zeros(batch_size)).cuda()

    onehot_imsize = Variable(torch.zeros(batch_size,10,32,32)).cuda()
    for ii in range(batch_size):
      onehot_imsize[ii][labels[ii]] = 1;

    ### train with real image
    prob = D(images,onehot_imsize)
    real_loss = criterion(prob,real_labels)
    #real_label은 1, 따라서 -r*log(p)-(1-r)*log(1-p)=-log(p)

    #log_prob = -torch.mean(torch.log(prob))

    ### train with fake image
    labels = Variable(torch.rand(batch_size)*10).type(torch.LongTensor).cuda()

    onehot = Variable(torch.zeros(batch_size,10,1,1)).cuda()
    onehot_imsize = Variable(torch.zeros(batch_size,10,32,32)).cuda()
    for ii in range(batch_size):
      onehot[ii][labels[ii]] = 1;
      onehot_imsize[ii][labels[ii]] = 1;
    
    fake_image = G(z,onehot)
    prob = D(fake_image,onehot_imsize)
    fake_loss = criterion(prob,fake_labels)
    #fake_label은 0, 따라서 -r*log(p)-(1-r)*log(1-p)=-log(1-p)

    d_loss = real_loss + fake_loss # real, fake에 대해 각각 1, 0을 만들도록
    d_optimizer.zero_grad()
    d_loss.backward()
    d_optimizer.step()

    ### Train G ###

    ### train with fake image
    #geneartor 학습시에는 또다른 z를 생성. 
    #좀전에 가짜라고 한걸 진짜라고 하는것보다 다른 z를 sampling 하는게 좋음
    z = Variable(torch.randn(batch_size, z_dim,1,1)).cuda()
    labels = Variable(torch.rand(batch_size)*10).type(torch.LongTensor).cuda()

    onehot = Variable(torch.zeros(batch_size,10,1,1)).cuda()
    onehot_imsize = Variable(torch.zeros(batch_size,10,32,32)).cuda()
    for ii in range(batch_size):
      onehot[ii][labels[ii]] = 1;
      onehot_imsize[ii][labels[ii]] = 1;

    fake_image = G(z,onehot)
    
    prob = D(fake_image,onehot_imsize)
    g_loss = criterion(prob,real_labels) # fake image를 real로 구분하도록

    g_optimizer.zero_grad()
    g_loss.backward()
    g_optimizer.step()

    if i % 100 == 0:
      print('Epoch %d, step %d, d_loss %.4f, g_loss %.4f'
            %(epoch+1,i+1,d_loss.data,g_loss.data));
    if( i % 500 == 0 ):
      fake_image = fake_image.view(batch_size,1, 32, 32)
      sample_path = 'samples'#fake_img'+str(i)+'.jpg'
      plt.figure(figsize=(5,10))
      for ii in range(32):
        plt.subplot(8,4,ii+1)
        plt.imshow(fake_image[ii][0].cpu().detach().numpy())
      plt.savefig(sample_path+'/fake_image%2d_%2d.png'%(epoch,i))
      plt.close("all")


  if (epoch+1) % 10 == 0 :
    if not os.path.exists('models'):
      os.makedirs('models')
    torch.save(D,'models/discriminator_%3d.pt'%(epoch+1))
    torch.save(G,'models/generator_%3d.pt'%(epoch+1))
       


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1, 1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 1, step 1, d_loss 0.0053, g_loss 6.5710
Epoch 1, step 101, d_loss 0.0831, g_loss 5.1011
Epoch 1, step 201, d_loss 0.0558, g_loss 6.4274
Epoch 1, step 301, d_loss 0.0778, g_loss 6.1448
Epoch 1, step 401, d_loss 0.0552, g_loss 4.3896
Epoch 1, step 501, d_loss 0.1058, g_loss 7.0507
Epoch 1, step 601, d_loss 0.1347, g_loss 6.6759
Epoch 1, step 701, d_loss 0.2880, g_loss 3.9941
Epoch 1, step 801, d_loss 0.0807, g_loss 4.1408
Epoch 1, step 901, d_loss 0.0657, g_loss 4.8515
Epoch 1, step 1001, d_loss 0.0171, g_loss 8.4541
Epoch 1, step 1101, d_loss 0.1989, g_loss 4.9191
Epoch 1, step 1201, d_loss 0.0802, g_loss 5.0471
Epoch 1, step 1301, d_loss 0.0811, g_loss 5.6621
Epoch 1, step 1401, d_loss 0.0252, g_loss 7.3079
Epoch 1, step 1501, d_loss 0.1597, g_loss 7.2185
Epoch 1, step 1601, d_loss 0.0565, g_loss 4.9688
Epoch 1, step 1701, d_loss 0.0282, g_loss 3.0613
Epoch 1, step 1801, d_loss 0.0175, g_loss 6.7097
Epoch 2, step 1, d_loss 0.0226, g_loss 5.6237
Epoch 2, step 101, d_loss 0.0159, g